MegaDetector version 5

In [1]:
import os
from PIL import Image
import numpy as np
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.data import transforms as pw_trans
from PytorchWildlife import utils as pw_utils
from tqdm import tqdm


In [ ]:
DEVICE = "cpu"
detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")

input_directory = "../Mask_RCNN/input_frames"
output_directory_annotated = "output_bbox_frames"
output_directory_cropped = "output_cropped_frames"
os.makedirs(output_directory_annotated, exist_ok=True)
os.makedirs(output_directory_cropped, exist_ok=True)

transform = pw_trans.MegaDetector_v5_Transform(
    target_size=detection_model.IMAGE_SIZE,
    stride=detection_model.STRIDE
)

for image_name in tqdm(os.listdir(input_directory)):
    if image_name.endswith(".png"):  # format
        input_path = os.path.join(input_directory, image_name)

        try:
            img = np.array(Image.open(input_path).convert("RGB"))
            results = detection_model.single_image_detection(transform(img), img.shape, input_path)

            if results and "detections" in results and results["detections"]:
                annotated_temp_dir = os.path.join(output_directory_annotated, "temp")
                cropped_temp_dir = os.path.join(output_directory_cropped, "temp")

                pw_utils.save_detection_images([results], annotated_temp_dir, overwrite=True)
                pw_utils.save_crop_images([results], cropped_temp_dir, overwrite=True)

                for temp_file in os.listdir(annotated_temp_dir):
                    temp_file_path = os.path.join(annotated_temp_dir, temp_file)
                    output_path = os.path.join(output_directory_annotated, image_name)
                    os.rename(temp_file_path, output_path)

                for temp_file in os.listdir(cropped_temp_dir):
                    temp_file_path = os.path.join(cropped_temp_dir, temp_file)
                    output_path = os.path.join(output_directory_cropped, image_name)
                    os.rename(temp_file_path, output_path)

                os.rmdir(annotated_temp_dir)
                os.rmdir(cropped_temp_dir)

            else:
                print(f"No valid detections found for: {image_name}")

        except Exception as e:
            print(f"Error processing {image_name}: {e}")

print("Processing complete.") 

Fusing layers... 
Fusing layers... 
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
 20%|██        | 2/10 [00:44<02:57, 22.17s/it]